Link para melhor visualização do Notebook em seu ambiente de execução: [Notebook Colab](https://colab.research.google.com/drive/1KsP5Daww-lenfJqeuoN4EsFCsghUEdTn?usp=sharing)


---


Notebook desenvolvido para realizar a construção da planilha de resultados dos modelos treinados para um determinado corpus (UFSC, CFH, HST, SAUDE-CORPO, etc). Em posse desta planilha pode-se realizar diversos tipos de análise e é um ótimo objeto de estudos e pesquisas para análise de dados do Repositório Institucional da UFSC além, claro, dos modelos em si.

Por meio da visualização dos dados obtidos na planilha pode-se perceber os modelos que melhor pontuaram nas analogias Gerais (agrupamento das analogias de todos os temas) e também nas analogias separadas por temas. Com isso pode-se analisar que parâmetros influenciaram mais para resultados melhores, quais analogias foram as mais acertadas pelos modelos treinados e quais foram as menos acertadas (isso pode dizer se há textos suficientes ou não falando daquele determinado assunto tratado nessas analogias).

O notebook anterior (de múltiplos treinamentos) já gerou arquivos de resultados para cada modelo referente as questões gerais, porém neste notebook também será feita a geração da planilha de analogias básicas e gerais (realizando o processo de validação novamente para os modelos treinados utilizando essas duas validações de analogias: básicas e gerais separadamente).

# Preparação/Configuração de ambiente

In [ ]:
try:
    from gensim.models import KeyedVectors
    import os
    import pandas as pd
    from openpyxl.styles import Font, PatternFill
    import msgpack
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    from google.colab import output
except Exception as e:
    erro = f'{e._class__.__name__}: {str(e)}'
    print(f'Erro ao configurar ambiente:\n--> {erro}')
else:
    print('Ambiente configurado com sucesso!')

try:
    import joblib
    import sys
    import os
    import pandas as pd
    from gensim.models import Word2Vec
    from google.colab import drive
    drive.mount('/content/drive')
    from google.colab import output
    import msgpack
    !pip install ferramentas-basicas-pln -U
    from ferramentas_basicas_pln import formatarTexto,removerCaracteresEspeciais,removerCaracteresEstranhos,removerEspacosEmBrancoExtras,transformarTextoSubstituindoCaracteres,coletarTextoDeArquivoTxt
    from ferramentas_basicas_pln import STRING_CARACTERES_ESPECIAIS_PADRAO
except Exception as e:
    erro = f'{e._class__.__name__}: {str(e)}'
    print(f'Erro ao configurar ambiente:\n--> {erro}')
else:
    print('Ambiente configurado com sucesso!')

Mounted at /content/drive
Ambiente configurado com sucesso!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ambiente configurado com sucesso!
Mounted at /content/drive
Ambiente configurado com sucesso!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ambiente configurado com sucesso!


# **GERAÇÃO DE PLANILHA**
Exec só quando os programas de treino finalizarem

## Teste com analogias básicas e gerais (modelos já treinados e armazenados aí roda-se essas células abaixo)

### Funções

In [ ]:
from gensim.models import Word2Vec
from typing import Generator

def obtemQuestoes(caminho_arquivo_txt_questoes : str = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/analogias_WOKE.txt',
                  caminho_arquivo_txt_contra_conceitos : str = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/contra-conceitos_WOKE.txt') -> dict:
  """
  Função responsável por obter e construir uma variável que contemple as
  questões/analogias para validação dos resultados relacionados à acurácia dos
  modelos treinados.

  ### Parâmetros:
  - caminho_arquivo_txt_questoes: String contendo o caminho até o txt com as questões
  relacionadas às analogias.
  - caminho_arquivo_txt_contra_conceitos: String contendo o caminho até o txt de
  questões relacionadas ao teste de captura de contra-conceitos. Esse arquivo é
  para testar uma ideia que apareceu para verificar se algum modelo treinado em
  determinados parâmetros seria capaz de capturar contra-conceitos sendo estes
  praticamente opostos ao vetor do seu conceito, ou seja, vetor do conceito estaria
  a, praticamente, 180 graus do vetor do contra-conceito. Depois de alguns testes
  foi observado que uma tática melhor era usar analogias de conceitos e contra-conceitos
  para tentar mensurar esse tipo de desempenho nos modelos treinados.

  ### Retornos:
  - Dicionário com as questões organizadas em "similaridade-positiva" (caso queiramos
  validar simplesmente os vizinhos mais próximos de detemrinados tokens), "analogias"
  para testar, de fato, as analogias, "similaridade-negativa" para fazer a verificação
  dos contra-conceitos. Cada parte dita anteriormente possuíra temas e dentro de
  cada tema terá uma lista com as "questões".
  """
  txt_questoes = coletarTextoDeArquivoTxt(caminho_arquivo=caminho_arquivo_txt_questoes,tipo_de_encoding='utf-8').lower()

  dic_questoes = {'similaridade-positiva':{},'analogia':{}}

  for linha in [line.strip() for line in txt_questoes.split('\n') if line.strip() != '']:
    if linha.startswith(':'):
      tema = linha[1:].strip()
      dic_questoes['analogia'][tema] = []
    else:
      questao = [palavra.strip() for palavra in linha.lower().split(',') if palavra.strip() != '']
      if len(questao) == 4:
        dic_questoes['analogia'][tema].append(questao)


  # Alternativa para capturar contra-conceitos
  txt_questoes_similaridade_negativa = coletarTextoDeArquivoTxt(caminho_arquivo=caminho_arquivo_txt_contra_conceitos,tipo_de_encoding='utf-8').lower()
  dic_questoes['similaridade-negativa'] = {'contra-conceitos':[]}
  for linha in [l.strip() for l in txt_questoes_similaridade_negativa.split('\n') if l.strip() != '']:
    questao = [palavra.strip() for palavra in linha.lower().split(',') if palavra.strip() != '']
    if len(questao) > 1:
      dic_questoes['similaridade-negativa']['contra-conceitos'].append(questao)

  return dic_questoes

def obtemQuestoesBasicas(caminho_arquivo_txt_questoes : str = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/analogias_basicas_WOKE.txt') -> dict:
  """
  Função responsável por obter e construir uma variável que contemple as
  questões/analogias consideradas básicas para validação dos resultados relacionados
  à acurácia dos modelos treinados nessas questões.

  ### Parâmetros:
  - caminho_arquivo_txt_questoes: String contendo o caminho até o arquivo
  de texto que consta as analogias básicas.

  ### Retornos:
  - Dicionário com as questões básicas organizadas em "similaridade-positiva" (caso
  queiramos validar simplesmente os vizinhos mais próximos de detemrinados tokens),
  "analogias" para testar, de fato, as analogias. Cada parte dita anteriormente possuíra temas
  e dentro de cada tema terá uma lista com as "questões".
  """
  txt_questoes = coletarTextoDeArquivoTxt(caminho_arquivo=caminho_arquivo_txt_questoes,tipo_de_encoding='utf-8').lower()

  dic_questoes = {'similaridade-positiva':{},'analogia':{}}

  for linha in [line.strip() for line in txt_questoes.split('\n') if line.strip() != '']:
    if linha.startswith(':'):
      tema = linha[1:].strip()
      dic_questoes['analogia'][tema] = []
    else:
      questao = [palavra.strip() for palavra in linha.lower().split(',') if palavra.strip() != '']
      if len(questao) == 4:
        dic_questoes['analogia'][tema].append(questao)

  return dic_questoes


def validarQuestoesBasicas(modelo,
                           incluido_no_topn : int = 5,
                           caminho_arquivo_txt_questoes_basicas : str = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/analogias_basicas_WOKE.txt',
                           silence : bool = True):
    """
    Função responsável por realizar a validação das questões básicas. Ou seja,
    retornará True caso o modelo acerte todas as questões básicas ou False caso
    erre alguma.

    ### Parâmetros:
    - modelo: Modelo Word2Vec a ser testado nas analogias básicas.
    - incluido_no_topn: Inteiro responsável por restringir a resposta das analogias.
    - caminho_arquivo_txt_questoes_basicas: String contendo o caminho até o arquivo
    de texto que consta as analogias básicas.
    - silence: Bool que printará (True) ou não (False) na tela de output se o modelo
    em questão errou ou não alguma analogia.

    ### Retornos:
    - Bool responsável por dizer se o modelo em questão acertou todas as analogias
    básicas (True) ou não (False).
    """
    txt_questoes_questoes_basicas = coletarTextoDeArquivoTxt(caminho_arquivo=caminho_arquivo_txt_questoes_basicas,tipo_de_encoding='utf-8').lower()
    for linha in [l.strip() for l in txt_questoes_questoes_basicas.split('\n') if l.strip() != '']:
      questao = [palavra.strip() for palavra in linha.lower().split(',') if palavra.strip() != '']
      # Testando analogia
      if len(questao) == 4:
        try:
          resultado = [r[0] for r in modelo.most_similar_cosmul(positive=[questao[1],questao[2]],negative=[questao[0]],topn=incluido_no_topn)]
          acertou = False
          for resposta_certa in questao[3].split('|'):
            if resposta_certa in resultado:
              acertou = True
              break
          if not acertou:
            if not silence:
              print(f'\nErrou: {str(questao)}, resultado: {str(resultado)}')
            return False
        except Exception as e:
          return False

      # # Testando similaridade positiva
      # if len(questao) == 2:
      #   try:
      #     resultado = [r[0] for r in modelo.wv.most_similar(questao[0],topn=incluido_no_topn)]
      #     acertou = False
      #     for resposta_certa in questao[1].split('|'):
      #       if resposta_certa in resultado:
      #         acertou = True
      #         break
      #     if not acertou:
      #       if not silence:
      #         print(f'\nErrou: {str(questao)}, resultado: {str(resultado)}')
      #       return False
      #   except Exception as e:
      #     return False


    # Caso o modelo acerte todas
    return True


def Comparacao(modelo, homem, rei, mulher):
  """
  Função responsável por realizar a comparação (analogia) e retornar o resultado.
  Os parâmetros estão pensados no seguinte tipo de analogia:

  homem   -->   rei
  mulher  -->    X

  Sendo X o resultado que deseja-se obter.

  ### Parâmetros:
  - modelo: Modelo Word2Vec o qual quer se extrair o resultado de alguma analogia.
  - homem: String que substituirá o token "homem" na analogia.
  - rei: String que substituirá o token "rei" na analogia.
  - mulher: String que substituirá o token "mulher" na analogia.

  ### Retornos:
  - Lista de strings em ordem de resultado (top 3) para a analogia dada como entrada.
  """
  try:
    resultado = [resultado[0] for resultado in modelo.most_similar_cosmul(positive=[rei,mulher],negative=[homem],topn=3)]
    return resultado
  except Exception as e:
    return []

def Similaridade(modelo, palavra):
  """
  Função responsável por realizar a obtenção dos vizinhos mais próximos de um token
  num determinado modelo.

  ### Parâmetros:
  - modelo: Modelo Word2Vec o qual quer se extrair o resultado dos vizinhos mais
  próximos do token/palavra dado como entrada.
  - palavra: String contendo o token/palavra que deseja-se buscar os vizinhos mais
  próximos.

  ### Retornos:
  - Lista de strings em ordem de resultado (top 3) para os vizinhos mais próximos
  do token dado como entrada.
  """
  try:
    resultado = [resultado[0] for resultado in modelo.most_similar(palavra,topn=3)]
    return resultado
  except Exception as e:
    return []

def SimilaridadeNegativaTestada(modelo, questao):
  """
  Função responsável por realizar o teste de similaridade negativa de uma questão
  num determinado modelo. A questão vem no formato ["token1","token2|token3"],
  onde "token1" é o token o qual será comparado à similaridade com os demais "token2"
  e "token3" (neste exemplo). Ou seja, será comparado a similaridade:
  similarity(token1,token2) e similarity(token1,token3) e os resultados retornados.

  ### Parâmetros:
  - modelo: Modelo Word2Vec o qual quer se extrair as similaridades entre os tokens
  dados na questão de entrada.
  - questao: Lista de dois elementos (do tipo string) sendo o primeiro elemento
  o token que deseja se obtem a similaridade entre os demais que estão no segundo
  elemento da lista (caso desejar comparar mais de um token separá-los pelo caracter
  "|". Exemplo: questao = ["historia","matemática|física"] ).

  ### Retornos:
  - Lista contendo as similaridades (na ordem dos tokens da questao) referentes
  à questão dada como entrada.
  """
  lista_similaridades = []
  for resposta_certa in questao[-1].split('|'):
    try:
      lista_similaridades.append(modelo.similarity(questao[0],resposta_certa))
    except Exception as e:
      pass
  return lista_similaridades

def validaResultadoSimilaridadeNegativaTestada(lista_respostas_similaridades : list[float],
                                               valor_minimo : float = -0.8) -> int:
  """
  Função responsável por verificar (retornando True/False) se o menor valor na
  lista de similaridades é menor do que o valor mínimo pré-setado (-0,8). Caso
  isso ocorra retorna True, caso contrário False.

  ### Parâmetros:
  - lista_respostas_similaridades: Lista de elementos do tipo float referente à
  lista de similaridades entre tokens.
  - valor_minimo: float (geralmente negativo) para comparar com o menor valor obtido
  na lista de similaridades.

  ### Retornos:
  - Bool: True se existir um valor menor que o valor mínimo na lista de similaridades,
  ou False caso não existir.
  """
  if min(lista_respostas_similaridades) < valor_minimo:
    return 3
  else:
    return 0

def SimilaridadeNegativaReal(modelo, palavra):
  """
  Função responsável por obter os top 3 vizinhos menos próximos (via similaridade)
  de uma palavra/token num determinado modelo.

  ### Parâmetros:
  - modelo: Modelo Word2Vec o qual se deseja extrair as informações.
  - palavra: String contendo o token que se deseja extrair os top 3 vizinhos menos
  próximos.

  ### Retornos:
  - Lista de strings contendo os top 3 vizinhos menos próximos já na ordem do menos
  próximo pro mais próximo.
  """
  max_size = len(modelo.wv.index_to_key)
  try:
    # Pode-se tentar usar topn=None para retornar a lista inteira de vizinhos mais próximos também
    resultado = [r[0] for r in modelo.wv.most_similar(palavra,topn=max_size)][-3:] # [-3:] pegamos somente os 3 últimos da lista (os "menos" similares ou seja, menos próximos de 1, mais próximos de -1)
    resultado.reverse()
    return resultado
  except Exception as e:
      return []

def ValidaResultado(gabarito : list, resposta : list):
  """
  Função responsável por fazer a validação de um resultado obtido de acordo com
  um gabarito esperado no resultado. O resultado pode ter mais de uma resposta,
  as quais serão comparadas com a resposta esperada do gabarito.

  ### Parâmetros:
  - gabarito: Lista de strings que seria o gabarito de uma questão.
  - resposta: Lista de strings obtida como resultado de uma questão.

  ### Retornos:
  - Inteiro 3, 1 ou 0 que faz referência à acerto total (algum resultado bateu com
  o gabarito), acerto parcial (o gabarito estava presente na segunda ou terceira
  posição do resultado) ou resposta errada (resultado não bateu com o gabarito),
  respectivamente.
  """
  if '|' in gabarito[-1]:
    respostas_certas = gabarito[-1].split('|')
    for resposta_certa in respostas_certas:
      if resposta_certa == resposta[0]:
        return 3
    for resposta_certa in respostas_certas:
      if resposta_certa in resposta[1:]:
        return 1
    return 0

  else:
    resposta_certa = gabarito[-1]
    if resposta_certa == resposta[0]:
      return 3
    elif resposta_certa in resposta[1:]:
      return 1
    return 0

def montarDicResultados(modeloW2V,
                        dic_questoes : dict,
                        basicas : bool = False) -> dict:
  """
  Função responsável por construir um dicionário de resultados para as questões.

  ### Parâmetros:
  - modeloW2V: Modelo Word2Vec que está se testando no momento.
  - dic_questoes: Dicionário contendo as questões que serão testadas/validadas.
  - basicas: Bool que dirá se a validação atual se trata (True) ou não (False) de
  analogias básicas.

  ### Retornos:
  - Dicionário contendo as questões e suas devidas pontuações para um determinado
  modelo.
  """
  dic_resultados = {'similaridade-positiva':{},'analogia':{}}

  for tema in dic_questoes['similaridade-positiva']:

    dic_resultados['similaridade-positiva'][tema] = {}

    for questao in dic_questoes['similaridade-positiva'][tema]:

      resultado = Similaridade(modeloW2V,questao[0])
      if resultado:
        pontuacao = ValidaResultado(questao,resultado)
      else:
        pontuacao = 0

      dic_resultados['similaridade-positiva'][tema][', '.join(questao).replace("'","")] = pontuacao

  for tema in dic_questoes['analogia']:

    dic_resultados['analogia'][tema] = {}

    for questao in dic_questoes['analogia'][tema]:

      resultado = Comparacao(modeloW2V,questao[0],questao[1],questao[2])
      if resultado:
        pontuacao = ValidaResultado(questao,resultado)
      else:
        pontuacao = 0

      dic_resultados['analogia'][tema][', '.join(questao).replace("'","")] = pontuacao

  if not basicas:
    dic_resultados['similaridade-negativa op1'] = {}

    for tema in dic_questoes['similaridade-negativa']:

      dic_resultados['similaridade-negativa op1'][tema] = {}

      for questao in dic_questoes['similaridade-negativa'][tema]:
        resultado = SimilaridadeNegativaReal(modeloW2V,questao[0])
        if resultado:
          pontuacao = ValidaResultado(questao,resultado)
        else:
          pontuacao = 0

        dic_resultados['similaridade-negativa op1'][tema][', '.join(questao).replace("'","")] = pontuacao

    dic_resultados['similaridade-negativa op2'] = {}

    for tema in dic_questoes['similaridade-negativa']:

      dic_resultados['similaridade-negativa op2'][tema] = {}

      for questao in dic_questoes['similaridade-negativa'][tema]:
        resultado = SimilaridadeNegativaTestada(modelo=modeloW2V,questao=questao)
        if resultado:
          pontuacao = validaResultadoSimilaridadeNegativaTestada(lista_respostas_similaridades=resultado)
        else:
          pontuacao = 0

        dic_resultados['similaridade-negativa op2'][tema][', '.join(questao).replace("'","")] = pontuacao

  return dic_resultados


def salvarResultadosEmMsgPack(nome_variavel : str,
                              variavel_em_questao,
                              pasta_para_salvar : str) -> tuple[bool,str]:
    """
    Função responsável por salvar, no formato "msgpack", alguma variável deste
    ambiente numa pasta especificada.

    ### Parâmetros:
    - nome_variavel: String contendo o nome da variável que será dado ao arquivo.
    - variavel_em_questao: Variável propriamente dita que se deseja salvar num
    arquivo.
    - pasta_para_salvar: String contendo o caminho até a pasta onde se deseja
    salvar o arquivo da variável em questão.

    ### Retornos:
    - Tupla com dois elementos. O primeiro do tipo bool que faz referência ao
    status do processo (True bem sucedido, False falhou) e o segundo do tipo
    string contendo a mensagem referente ao status obtido.
    """
    try:
        # Se a pasta não existir, efetuamos sua criação
        if not os.path.exists(pasta_para_salvar):
            os.makedirs(pasta_para_salvar)

        # Obtendo os bytes da variável que queremos salvar
        variable_bytes = msgpack.packb(variavel_em_questao)

        # Se o nome do arquivo que salvará a variável não tiver o formato msgpack ao final, iremos adicioná-lo.
        if not nome_variavel.endswith('.msgpack'):
            nome_variavel += '.msgpack'

        # Salvando a variável desejada em bytes no formato .msgpack
        with open(os.path.join(pasta_para_salvar,nome_variavel),'wb') as f:
            f.write(variable_bytes)
            f.close()
            return True, f'Variável {nome_variavel} salva com sucesso no formato .msgpack'

    # Caso ocorra algum erro inesperado, trataremos o enviando como mensagem de falha juntamente com um status False (de falha/erro no processo)
    except Exception as e:
        error_message = f'{e.__class__.__name__}: {str(e)}'
        return False, error_message

def abrirArquivoMsgPack(full_filepath : str,
                        encoding_type : str = None):
    """
    Função responsável por abrir os arquivos no formato msgpack.

    ### Parâmetros:
    - full_filepath: String contendo o caminho completo até o arquivo que deseja-se
    abrir e extrair o conteúdo (variável salva).
    - encoding_type: String contendo o tipo de encoding, caso desejar.

    ### Retornos:
    - Variável salva (e agora aberta e lida) no arquivo msgpack.
    """
    if not full_filepath.endswith('.msgpack'):
        full_filepath += '.msgpack'
    if encoding_type:
        with open(full_filepath,'rb',encoding=encoding_type) as f:
            variable_bytes = f.read()
            variable_loaded = msgpack.unpackb(variable_bytes, raw=False)
            f.close()
            return variable_loaded
    else:
        with open(full_filepath,'rb') as f:
            variable_bytes = f.read()
            variable_loaded = msgpack.unpackb(variable_bytes, raw=False)
            f.close()
            return variable_loaded

def organizarAmbienteDeTreinos(nome_pasta_treino : str,
                               escopo_de_treinamento : tuple[int,int],
                               reset_arquivo_txt_mapa_treinos : bool = False,
                               caminho_pasta_para_arquivo_txt_mapa_treinos : str = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/Resultados Múltiplos Treinamentos') -> tuple[bool,str]:
  """
  Função responsável por efetuar a organização das pastas para deixar o ambiente
  preparado para salvar as pastas e os arquivos de treinamento.

  ### Parâmetros:
  - nome_pasta_treino: String contendo o nome da pasta de treino.
  - escopo_de_treinamento: Tupla de dois elementos sendo o primeiro uma lista das
  coleções que serão contempladas neste treinamento e o segundo uma outra tupla
  de dois inteiros do ano inicial e ano final, respectivamente, para o treinamento.
  - reset_arquivo_txt_mapa_treinos: Bool que dirá se deve-se apagar ou não o arquivo
  de informação dos treinos já efetuados.
  - caminho_pasta_para_arquivo_txt_mapa_treinos: Caminho até o corpus pré-processado
  onde será buscado as coleções, anos, trabalhos e arquivos de pré-processamento
  para alimentação do treinamento.

  ### Retornos:
  - Tupla com elemento do tipo bool (status do processo) e elemento do tipo string
  sendo uma mensagem referente ao status (True deu tudo certo e False ocorreu um
  erro).
  """
  caminho_arquivo_txt_mapa_treinos = os.path.join(caminho_pasta_para_arquivo_txt_mapa_treinos,'Mapa dos Treinos.txt')
  try:
    string_escopo_treino = '--> Coleções:\n'+'\n'.join(escopo_de_treinamento[0])+f"\n--> Datas contempladas:\nDe {escopo_de_treinamento[1][0]} até {escopo_de_treinamento[1][1]}.\n\n"
    if (not os.path.exists(caminho_arquivo_txt_mapa_treinos)) or reset_arquivo_txt_mapa_treinos:
      with open(caminho_arquivo_txt_mapa_treinos,'w',encoding='utf-8') as f:
          f.write('='*(len('MAPA DE TREINOS')+4)+'\n'+'  MAPA DE TREINOS  \n'+'='*(len('MAPA DE TREINOS')+4)+'\n\n')
    if not os.path.exists(caminho_arquivo_txt_mapa_treinos.replace(os.path.basename(caminho_arquivo_txt_mapa_treinos),nome_pasta_treino)):
      os.makedirs(caminho_arquivo_txt_mapa_treinos.replace(os.path.basename(caminho_arquivo_txt_mapa_treinos),nome_pasta_treino))
      with open(caminho_arquivo_txt_mapa_treinos,'a',encoding='utf-8') as f:
          f.write('\n'+nome_pasta_treino+'\n'+'-'*100+'\n'+string_escopo_treino+'\n\n')
    return True, ''
  except Exception as e:
    return False, e.__class__.__name__ + ': ' +str(e)

def validarResultados(modeloW2V,
                      nome_modelo : str,
                      dic_questoes : dict,
                      pasta_para_salvar_resultados : str,
                      basicas : bool = False) -> dict:

  """
  Função responsável por obter e salvar os resultados para as questões de um determinado
  modelo.

  ### Parâmetros:
  - modeloW2V: Modelo Word2Vec que deseja-se validar os resultados para as questões.
  - nome_modelo: String contendo o nome do modelo que está se testando/validando.
  - dic_questoes: Dicionário contendo as questões que serão testadas.
  - pasta_para_salvar_resultados: String contendo o caminho até a pasta que será
  salvo os resultados para as questões do determinado modelo em questão.

  ### Retornos:
  - Tupla de dois elementos sendo o primeiro o status da validação de resultados
  (True se ocorreu tudo certo e False caso tenha ocorrido algum erro) e o segundo
  uma string contendo a mensagem referente ao status do processo (em caso de falha
  retorna o erro junto).
  """
  dic_resultados = montarDicResultados(modeloW2V=modeloW2V,dic_questoes=dic_questoes,basicas=basicas)

  if not nome_modelo.endswith('.msgpack'):
    nome_modelo += '.msgpack'

  status_save, msg_save = salvarResultadosEmMsgPack(nome_variavel=nome_modelo,variavel_em_questao=dic_resultados,pasta_para_salvar=pasta_para_salvar_resultados)

  if status_save:
    return True,'Resultados do treino salvos com sucesso!'
  else:
    return False,'\n! Problema ao salvar resultados do treino: '+msg_save+'\n'


def listagemDeParametros(lista_de_trabalhos : list,
                        n_programa : int,
                        numero_max_programas : int = 33) -> list:
  """
  Função responsável por dividir os conjuntos de parâmetros de um determinado treino
  nos programas que irão executar estes códigos.

  ### Parâmetros:
  - lista_de_trabalhos: Lista de dicionários que conterão os parâmetros de um determinado
  treino.
  - n_programa: Inteiro referente ao número do programa que está sendo executado.
  - numero_max_programas: Inteiro referente ao número total de programas que irão
  executar estes códigos.

  ### Retornos:
  - Lista de dicionários referentes aos conjuntos de parâmetros de treino que este
  programa irá executar.
  """
  dic_programas_trabalhos = {}
  for i in range(1,numero_max_programas+1):

    dic_programas_trabalhos[f'Prog {i}'] = []

  for i in range(len(lista_de_trabalhos)):
    if (i+1) <= numero_max_programas:
      num = i + 1
      dic_programas_trabalhos[f'Prog {num}'].append(lista_de_trabalhos[i])
    else:
      if (i+1)%numero_max_programas != 0:
        num = (i+1)%numero_max_programas
        dic_programas_trabalhos[f'Prog {num}'].append(lista_de_trabalhos[i])
      else:
        num = numero_max_programas
        dic_programas_trabalhos[f'Prog {num}'].append(lista_de_trabalhos[i])
  return dic_programas_trabalhos[f'Prog {n_programa}']

def criarInfoPlanilha(lista_de_dicionarios : list) -> dict:
    """
    Função responsável por criar um dicionário que contemplará a estrutura que depois
    será passada para a função geradora do Excel. Portanto, esta função agrupa
    e estrutura todas as informações para posterior implementação na planilha.

    ### Parâmetros:
    - lista_de_dicionarios: Lista de strings que fazem referência aos caminhos até
    os modelos treinados.

    ### Retornos:
    - Dicionário contendo todas as informações de modelos treinados com seus respectivos
    pontos nas suas respectivas analogias, somatório de pontos, etc agrupadas e
    estruturadas no formato esperado para função geradora da planilha Excel.
    """
    if lista_de_dicionarios:
        planilha = {'Geral':{}}

        for caminho_dic in lista_de_dicionarios:
            nome_modelo = os.path.basename(caminho_dic)
            dic = abrirArquivoMsgPack(caminho_dic)
            lista_perguntas_modelo_atual = []
            lista_pontos_modelo_atual = []
            passou_no_teste_basico = True
            for tipo in dic:
                for tema in dic[tipo]:
                    lista_perguntas_atual = []
                    lista_pontuacao_atual = []
                    for pergunta in dic[tipo][tema]:
                        lista_pontuacao_atual.append(dic[tipo][tema][pergunta])
                        lista_perguntas_atual.append(pergunta)
                    if tipo + ' - ' + tema in planilha.keys():
                        planilha[tipo+' - '+tema][nome_modelo] = {'perguntas':lista_perguntas_atual,'respostas':lista_pontuacao_atual}
                    else:
                        planilha[tipo+' - '+tema] = {nome_modelo:{'perguntas':lista_perguntas_atual,'respostas':lista_pontuacao_atual}}
                    lista_pontos_modelo_atual += lista_pontuacao_atual
                    lista_perguntas_modelo_atual += lista_perguntas_atual
            planilha['Geral'][nome_modelo] = {'perguntas':lista_perguntas_modelo_atual,'respostas':lista_pontos_modelo_atual}

    return planilha

def criarExcel(info_planilha,
               caminho_pasta_planilhas : str,
               nome_da_planilha : str = 'RESULTADOS_MULTIPLOS_TREINAMENTOS.xlsx') -> tuple[bool, str]:
    """
    Função responsável por gerar uma planilha Excel dos dados validados nos treinamentos
    realizados para determinado corpus de treinamento.

    ### Parâmetros:
    - info_planilha: Dicionário estruturado com todos os dados de validação dos
    modelos treinados e das suas pontuações nas analogias.
    - caminho_pasta_planilhas: String contendo o caminho até a pasta que armazenará
    a planilha.
    - nome_da_planilha: String contendo o nome que o arquivo .xlsx receberá.

    ### Retornos:
    - Tupla com dois elementos. O primeiro do tipo bool que faz referência ao
    status do processo (True bem sucedido, False falhou) e o segundo do tipo
    string contendo a mensagem referente ao status obtido (caso exista erro a
    este será embutido na mensagem).
    """
    fill_vermelho_suave = PatternFill(start_color="ff5757", end_color="ff5757", fill_type="solid")
    fill_verde_suave = PatternFill(start_color="57ff57", end_color="57ff57", fill_type="solid")
    fill_amarelo_suave = PatternFill(start_color="ffffab", end_color="ffffab", fill_type="solid")

    try:
        with pd.ExcelWriter(os.path.join(caminho_pasta_planilhas, nome_da_planilha), engine='openpyxl') as writer:
            for chave_primaria, modelos in info_planilha.items():
                # print('CP',chave_primaria,'\n','modelos','\n',modelos,'\n\n')
                all_data = []

                resultados = ['Resultado:']
                for modelo in modelos:
                    resultados.append(sum(modelos[modelo]['respostas']))
                    # print('CP',chave_primaria,'modelo',modelo,'soma',sum(modelos[modelo]['respostas']))
                all_data.append(resultados)

                all_data.append(['Questões'] + list(modelos.keys()))

                for i in range(len(modelos[list(modelos.keys())[0]]['perguntas'])):
                    linha = [modelos[list(modelos.keys())[0]]['perguntas'][i]]
                    for modelo in modelos:
                        linha.append(modelos[modelo]['respostas'][i])
                    all_data.append(linha)

                df = pd.DataFrame(all_data)

                df.to_excel(writer, index=False, sheet_name=chave_primaria)

                worksheet = writer.sheets[chave_primaria]
                lista_top_values = []
                for r_idx, row in df.iterrows():
                    if r_idx not in [0,1]:
                        for c_idx, value in enumerate(row, start=1):
                            cell = worksheet.cell(row=r_idx + 2, column=c_idx)
                            if value == 0:
                                cell.fill = fill_vermelho_suave
                                # cell.font = Font(bold=True, size=10, name='Arial')
                            elif value == 1:
                                cell.fill = fill_amarelo_suave
                            elif value == 3:
                                cell.fill = fill_verde_suave
                    elif r_idx == 0:
                        lista_ptos = row.values.tolist()
                        lista_ptos.remove('Resultado:')
                        lista_top_values = sorted(lista_ptos,reverse=True)[:3]
                        # print('Lista de pontos',lista_ptos)
                        # print('Lista top3 pontos',lista_top_values)
                        for c_idx, value in enumerate(row, start=1):
                            cell = worksheet.cell(row=r_idx + 2, column=c_idx)
                            if value in lista_top_values:
                                cell.fill = fill_verde_suave
                                if value == lista_top_values[0]:
                                    cell.font = Font(bold=True,name='Calibri')


        return True, 'Planilha gerada com sucesso!'
    except Exception as e:
        erro = f'Erro "{e.__class__.__name__}": {str(e)}'
        return False, erro


### Execução

- Criação da pasta de resultados básicos e gerais
- Gera os arquivos de resultados para as questões básicas e gerais e salva nas pastas respectivas

In [ ]:
# Seleção da pasta de treinamentos realizados que se deseja fazer a validação
pasta_modelos_treinados = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/Resultados Múltiplos Treinamentos/Saude-do-Corpo-03-10'

# Selecionando a pasta que será alocado as validações para as analogias básicas
pasta_para_salvar_resultados_basicas = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/Resultados Múltiplos Treinamentos/Saude-do-Corpo-03-10 - Resultados básicas'
if not os.path.exists(pasta_para_salvar_resultados_basicas):
  os.makedirs(pasta_para_salvar_resultados_basicas)

# Selecionando a pasta que será alocado as validações para as analogias gerais
pasta_para_salvar_resultados_gerais = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/Resultados Múltiplos Treinamentos/Saude-do-Corpo-03-10 - Resultados gerais'
if not os.path.exists(pasta_para_salvar_resultados_gerais):
  os.makedirs(pasta_para_salvar_resultados_gerais)

# Obtendo os modelos que foram treinados
lista_modelos = [os.path.join(pasta_modelos_treinados,modelo) for modelo in os.listdir(pasta_modelos_treinados) if modelo.endswith('.model')]

# Obtendo a quantidade de modelos treinados
qtd_modelos = len(lista_modelos)


########################### Primeira etapa da execução para as analogias gerais ###########################


# Obtendo o dicionário de questões para realização das validações dos modelos treinados
dic_questoes_gerais = obtemQuestoes()

# Variáveis que contabilizarão os arquivos de validação bem sucedidos e mal sucedidos
msg_save_true = 0
msg_save_false = 0


for cont,caminho_modelo in enumerate(lista_modelos):
  output.clear()
  print('Obtendo analogias gerais',cont+1,'de',qtd_modelos)
  nome_modelo_atual = os.path.basename(caminho_modelo).replace('.model','')
  print(nome_modelo_atual)

  if not os.path.exists(os.path.join(pasta_para_salvar_resultados_gerais,nome_modelo_atual+'.msgpack')): # Se não foi catalogado arquivo de resultados para o modelo atual ainda
    # Carregar o tipo WordVectors do modelo atual (objeto mais leve do que o modelo todo), que já é suficiente para executar as operações necessárias nesta validação
    modelo_atual = Word2Vec.load(caminho_modelo).wv

    # Chamada da função que se encarrega de executar os processos para realização da validação e geração de arquivo de resultados
    status_validacao, msg_validacao = validarResultados(nome_modelo=nome_modelo_atual,
                                                        modeloW2V=modelo_atual,
                                                        pasta_para_salvar_resultados=pasta_para_salvar_resultados_gerais,
                                                        dic_questoes=dic_questoes_gerais)
  else:
    print('Arquivo encontrado para', nome_modelo_atual)
    status_validacao = True

  if status_validacao:
    msg_save_true += 1
  else:
    msg_save_false += 1

msg_1 = f'Analogias gerais, save_true: {msg_save_true} save_false: {msg_save_false}.\n'


########################### Segunda etapa da execução para as analogias básicas ###########################
# Segue a mesma lógica que a primeira etapa descrita à cima...

dic_questoes_basicas = obtemQuestoesBasicas()

msg_save_true = 0
msg_save_false = 0

for cont,caminho_modelo in enumerate(lista_modelos):
  output.clear()
  print('Obtendo analogias básicas',cont+1,'de',qtd_modelos)
  nome_modelo_atual = os.path.basename(caminho_modelo).replace('.model','')

  if not os.path.exists(os.path.join(pasta_para_salvar_resultados_basicas,nome_modelo_atual+'.msgpack')):

    modelo_atual = Word2Vec.load(caminho_modelo).wv

    status_validacao, msg_validacao = validarResultados(nome_modelo=nome_modelo_atual,
                                                        modeloW2V=modelo_atual,
                                                        pasta_para_salvar_resultados=pasta_para_salvar_resultados_basicas,
                                                        dic_questoes=dic_questoes_basicas, basicas = True)
  else:
    print('Arquivo encontrado para', nome_modelo_atual)
    status_validacao = True

  if status_validacao:
    msg_save_true += 1
  else:
    msg_save_false += 1

msg_2 = f'Analogias básicas, save_true: {msg_save_true} save_false: {msg_save_false}.\n'


output.clear()
print(msg_1,msg_2,sep='\n')

Analogias gerais, save_true: 1260 save_false: 0.

Analogias básicas, save_true: 1260 save_false: 0.



### Execução

- Geração de planilha para a determinada pasta: básicas ou gerais

In [ ]:
### Geração de planilha para resultados das questões gerais
caminho_pasta_resultados_do_treinamento = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/Resultados Múltiplos Treinamentos/Saude-do-Corpo-03-10 - Resultados gerais'

lista_caminho_dicionarios = [os.path.join(caminho_pasta_resultados_do_treinamento,d) for d in os.listdir(caminho_pasta_resultados_do_treinamento) if d.endswith('.msgpack')]
planilha = criarInfoPlanilha(lista_caminho_dicionarios)

print('Criação de planilha para os resultados das analogias gerais:')
print(criarExcel(info_planilha=planilha,caminho_pasta_planilhas=caminho_pasta_resultados_do_treinamento,nome_da_planilha=f'RESULTADOS_MULTIPLOS_TREINAMENTOS_gerais.xlsx'))

### Geração de planilha para resultados das questões básicas
caminho_pasta_resultados_do_treinamento = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/Resultados Múltiplos Treinamentos/Saude-do-Corpo-03-10 - Resultados básicas'

lista_caminho_dicionarios = [os.path.join(caminho_pasta_resultados_do_treinamento,d) for d in os.listdir(caminho_pasta_resultados_do_treinamento) if d.endswith('.msgpack')]
planilha = criarInfoPlanilha(lista_caminho_dicionarios)

print('Criação de planilha para os resultados das analogias básicas:')
print(criarExcel(info_planilha=planilha,caminho_pasta_planilhas=caminho_pasta_resultados_do_treinamento,nome_da_planilha=f'RESULTADOS_MULTIPLOS_TREINAMENTOS_básicas.xlsx'))

Criação de planilha para os resultados das analogias gerais:


/usr/local/lib/python3.10/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


(True, 'Planilha gerada com sucesso!')
Criação de planilha para os resultados das analogias básicas:
(True, 'Planilha gerada com sucesso!')


## Checagem da quantidade de arquivos de treinos e de resultados (em desuso atualmente)

In [ ]:
pasta_treinamentos = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/Resultados Múltiplos Treinamentos/CFH - 2003 - 2005'
pasta_resultados = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/Resultados Múltiplos Treinamentos/CFH - 2003 - 2005 - Resultados'

qtd_treinos = len([arq for arq in os.listdir(pasta_treinamentos) if arq.endswith('.model')])
qtd_treinos_set = len(set([arq for arq in os.listdir(pasta_treinamentos) if arq.endswith('.model')]))
qtd_resultados = len([arq for arq in os.listdir(pasta_resultados) if arq.endswith('.msgpack')])
qtd_resultados_set = len(set([arq for arq in os.listdir(pasta_resultados) if arq.endswith('.msgpack')]))

print(qtd_treinos,qtd_treinos_set)

print(qtd_resultados,qtd_resultados_set)

2396 2396
2402 2402


### Possível ajuste

In [ ]:
lista_caminho_modelos = []
for arquivo in [os.path.join(pasta_treinamentos,arq) for arq in os.listdir(pasta_treinamentos) if arq.endswith('.model')]:
  if os.path.basename(arquivo).replace('.model','') not in [arq.replace('.msgpack','') for arq in os.listdir(pasta_resultados) if arq.endswith('.msgpack')]:
    lista_caminho_modelos.append(arquivo)

print('Encontrei',len(lista_caminho_modelos))

def gerarArquivoResultados2(lista_caminhos_modelo : list,
                           n_programa : int,
                           caminho_pasta_para_salvar : str = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/Resultados Múltiplos Treinamentos/CFH - 2003 - 2005 - Resultados'):
    dic_questoes = obtemQuestoes()

    # lista_caminhos_modelo = listagemDeTarefas(lista=lista_caminhos_modelo,n_programa=n_programa)

    qtd_modelos = len(lista_caminhos_modelo)

    for cont,caminho_modelo in enumerate(lista_caminhos_modelo):
      print(cont+1,'de',qtd_modelos)
      try:
        # A lista de dicionários vai conter tuplas do tipo (nome_modelo,modelo)
        # lista_de_dicionarios.append((os.path.basename(caminho_modelo).replace('.model',''),validarResultados(modeloW2V=Word2Vec.load(caminho_modelo).wv,dic_questoes=dic_questoes)))
        if not os.path.exists(os.path.join(caminho_pasta_para_salvar,os.path.basename(caminho_modelo.replace('.model','.msgpack')))):
          dic_resultados_modelo_atual = validarResultados(modeloW2V=Word2Vec.load(caminho_modelo).wv,dic_questoes=dic_questoes)
          print(salvarResultadosEmMsgPack(os.path.basename(caminho_modelo.replace('.model','.msgpack')),dic_resultados_modelo_atual,caminho_pasta_para_salvar))
      except Exception as e:
        print(f'\n{e.__class__.__name__}: {str(e)}\n')
      else:
        print(f'Respostas de "{os.path.basename(caminho_modelo)}" catalogadas.')

gerarArquivoResultados2(lista_caminho_modelos,n_programa)

Encontrei 760
1 de 760
(True, 'Variável modelo_modo_0_dimensao_250_negative_5_window_4_epochs_5_alpha_0.01_min_count_80.msgpack salva com sucesso no formato .msgpack')
Respostas de "modelo_modo_0_dimensao_250_negative_5_window_4_epochs_5_alpha_0.01_min_count_80.model" catalogadas.
2 de 760
(True, 'Variável modelo_modo_0_dimensao_250_negative_5_window_4_epochs_8_alpha_0.025_min_count_65.msgpack salva com sucesso no formato .msgpack')
Respostas de "modelo_modo_0_dimensao_250_negative_5_window_4_epochs_8_alpha_0.025_min_count_65.model" catalogadas.
3 de 760
(True, 'Variável modelo_modo_0_dimensao_250_negative_15_window_8_epochs_5_alpha_0.025_min_count_65.msgpack salva com sucesso no formato .msgpack')
Respostas de "modelo_modo_0_dimensao_250_negative_15_window_8_epochs_5_alpha_0.025_min_count_65.model" catalogadas.
4 de 760
(True, 'Variável modelo_modo_0_dimensao_250_negative_10_window_8_epochs_5_alpha_0.025_min_count_100.msgpack salva com sucesso no formato .msgpack')
Respostas de "model